<div >
    <img src = "../banner/banner_ML_UNLP_1900_200.png" />
</div>

<a target="_blank" href="https://colab.research.google.com/github/ignaciomsarmiento/BDML_202501/blob/main/Modulo03/CuadernoModulo03_Regularizacion.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>



# Regularization

## Predicting Wages

Our objective today is to construct a model of individual wages

$$
w = f(X) + u 
$$

where w is the  wage, and X is a matrix that includes potential explanatory variables/predictors. In this tutorial set, we will focus on a linear model of the form

\begin{align}
 ln(w) & = \beta_0 + \beta_1 X_1 + \dots + \beta_p X_p  + u 
\end{align}

were $ln(w)$ is the logarithm of the wage.

To illustrate I'm going to use a sample of the NLSY97. The NLSY97 is  a nationally representative sample of 8,984 men and women born during the years 1980 through 1984 and living in the United States at the time of the initial survey in 1997.  Participants were ages 12 to 16 as of December 31, 1996.  Interviews were conducted annually from 1997 to 2011 and biennially since then.  

Let's load the packages and the data set:

In [ ]:
#install.packages("pacman") #for google colab

In [ ]:
#packages
require("pacman")
p_load("tidyverse","stargazer")

nlsy <- read_csv('https://raw.githubusercontent.com/ignaciomsarmiento/datasets/main/nlsy97.csv')

nlsy <- nlsy  %>%   drop_na(educ) #drops NAs

We want to construct a model that predicts well out of sample, and we have potentially 993 regressors. We are going to regularize this regression using the `glmnet` package.


## Glmnet

To apply a regularized model, we can use the `glmnet::glmnet()` function. `glmnet` solves the problem

$$
\min_{\beta_0,\beta} \frac{1}{N} \sum_{i=1}^{N} w_i l(y_i,\beta_0+\beta' x_i) + \lambda\left[(1-\alpha)\frac{1}{2}\|\beta\|_2^2 + \alpha \|\beta\|_1\right],
$$

where $l(y_i,\beta_0+\beta^T x_i)$ in the regression case is $\frac{1}{2}(y_i-\beta_0-\beta' x_i)$

The `alpha` parameter tells `glmnet` to perform a ridge (`alpha` = 0), lasso (`alpha` = 1), or elastic net (0 < `alpha` < 1) model. 

By default, `glmnet` will do two things that you should know:

1.  By default, `glmnet` automatically standardizes your features. If you standardize your predictors prior to glmnet you can turn this argument off with `standardize = FALSE`.

2. The regularization path is computed at a grid of values (on the log scale) for the regularization parameter $\lambda$. The algorithm is extremely fast!

`glmnet` has some drawbacks, the main one is that we need to specify the arguments in terms of matrices and vectors

`caret`, in contrast, streamlines the process of creating predictive models by providing a uniform interface for predictive models, which, among other things, allows for specifying formulas.

## Ridge

We first illustrate ridge regression, which can be fit using `glmnet()` with alpha = 0 and seeks to minimize

$$
\frac{1}{2n}\sum_{i=1}^{n} \left( y_i - \beta_0 - \sum_{j=1}^{p} \beta_j x_{ij}    \right) ^ 2 + \lambda \frac{1}{2}\sum_{j=1}^{p} \beta_j^2 .
$$

Notice that the intercept is not penalized. Why?


Ridge penalizes the squares  of the coefficients. As a result, ridge shrinks coefficients toward zero, but not all the way.


To understand the mechanics, we are going to focus on a small subset of variables:

1. **educ (Education Level)**  
   - This typically refers to the respondent's **highest level of education completed**.  
   - It is measured in **years of schooling** 
 
2. **mom_educ (Mother’s Education Level)**

- This represents the highest level of education completed by the respondent’s mother.
- Like "educ," it can be measured in years of schooling 
  

In [ ]:
p_load("glmnet")

#Vector that needs predicting
y <- nlsy$lnw_2016

# Matrix of predictors 
Xsmall <- as.matrix(nlsy  %>% select(educ,mom_educ))

Let's run the ridge regression (we need to set the parameter `alpha` to zero)

We solve:

$$
\min_{\beta_0,\beta_1,\beta_2} \frac{1}{2n} \sum_{i=1}^{n} \left( log(wage)_i - \beta_0 -  \beta_1 Educ_i - \beta_2 MomEduc_i   \right) ^ 2 + \lambda \frac{1}{2} \left(\beta_1^2+ \beta_2^2 \right)
$$

In [ ]:
ridge1 <- glmnet(
  x = Xsmall,
  y = y,
  lambda=1,
  alpha = 0 #ridge
)

In [ ]:
coef(ridge1)

Are these shrunken relative to OLS?

In [ ]:
stargazer(lm(y~Xsmall),type="text")

H.W. Why the intercepts are not equal? Tip: Think on how the intercept is computed.

### Regularization Paths

Let's see the regularization path, that shows how much the coefficients are penalized for different values of $\lambda$. 

In [ ]:
ridge2 <- glmnet(
  x = Xsmall,
  y = y,
  alpha = 0 #ridge
)

In [ ]:
plot(ridge2, xvar = "lambda")

Notice none of the coefficients are forced to be zero, although they get close to it.

#### All the predictors

In [ ]:
# Matrix of predictors (all but lnw_2016)
X <- as.matrix(nlsy  %>% select(-lnw_2016))

ridge_all <- glmnet(
  x = X,
  y = y,
  alpha = 0 #ridge
)

plot(ridge_all, xvar = "lambda")


The *regularization path*, which illustrates how the model’s coefficients change as we vary the regularization strength $\lambda$. Specifically:

1. **Penalizing Coefficients:**  
   When $\lambda$ is large, the penalty on the size of the coefficients is strong, pushing most coefficients toward zero (or shrinking them substantially). This helps prevent overfitting by limiting how large the coefficients are.

2. **Relaxing the Penalty:**  
   As $\lambda$ decreases, the penalty weakens. Coefficients that were previously shrunk toward zero may increase in magnitude if they improve the model’s fit to the data. Consequently, the model becomes more flexible and potentially more prone to overfitting.

3. **Visualizing the Path:**  
   We can plot each coefficient across a range of $\lambda$ values. Each line on the plot corresponds to a coefficient, showing how it changes from near-zero (with high $\lambda$) to larger values (with lower $\lambda$).

Overall, analyzing this path provides insight into the trade-off between regularization (controlling overfitting) and model complexity.

### Scale Equivariance

Since regularized methods apply a penalty to the coefficients, we need to ensure our coefficients are on a common scale. If not, then predictors with naturally larger values  will be penalized more than predictors with naturally smaller values.



Let's run the ridge regression (we need to set the parameter `alpha` to zero)

In [ ]:
ridge <- glmnet(
  x = Xsmall,
  y = y,
  alpha = 0, #ridge
  lambda=0,
  standardize=FALSE,
)

Let's see the coefficients we obtained


In [ ]:
coef(ridge)

Compare to OLS

In [ ]:
ols<-lm(y~Xsmall)
stargazer(ols,type="text")

#### What happens if we change the scale for mother's education?

In [ ]:
Xsmall[,2]<-Xsmall[,2]/10 #divide by 10, put it in decades

In [ ]:
ridge_10 <- glmnet(
  x = Xsmall,
  y = y,
  alpha = 0, #ridge
  lambda=20,
  standardize=FALSE,
)

In [ ]:
coef(ridge_10)

The original in years

In [ ]:
coef(ridge)[3]

In [ ]:
ols_10<-lm(y~Xsmall)
summary(ols_10)

In [ ]:
ols_10$coefficients[3]/10

In [ ]:
ols$coefficients[3] #the original

Does it work if we reescale the ridge coefficient?

In [ ]:
coef(ridge_10)[3]/10

#### A Bit of intuition

$$
\tilde{MomEduc}_i = \frac{MomEduc_i}{10}
$$

Now the model becomes:

$$
log(wage)_i = \beta_0 +  \beta_1 Educ_i + \tilde{\beta}_2 \tilde{MomEduc}_i + u_i
$$

where $\tilde{\beta}_2=10 \beta_2$

Substituting into the ridge penalty term:

$$
\lambda (\beta_1^2 + \beta_2^2) = \lambda \left( \beta_1^2 + \left(\frac{\tilde{\beta}_2}{10}\right)^2 \right)
$$

which simplifies to:

$$
\lambda \beta_1^2 + \lambda \frac{\tilde{\beta}_2^2}{100}
$$

This means that **the penalty on $\beta_2$ is scaled down by a factor of 100** when the variable is rescaled. In other words:

- **Before scaling**, the mother's education variable is shrunken at the same rate as years of education because they are on the same scale.
- **After scaling**, the shrinkage is relatively **smaller** because the coefficient is now divided by a factor.


Take aways:

1. **Magnitude Matters:**  
   - The ridge penalty **shrinks large coefficients more** because it treats all variables equally in regularization.
   - Thus, if mother's education is measured in years, its coefficient will be **larger** than if measured in decades, leading to a **stronger penalization**.

2. **Standardization Helps:**  
   - In practice, ridge regression often **standardizes variables** to avoid uneven penalization.
   - Standardizing ensures that all variables have a mean of 0 and variance of 1, making ridge penalties apply evenly. Variables now are unitless.



### Penalty selection

Now we need to discuss how we select $\lambda$: 

- Making sure that we have **zero bias** within the sample creates **out-of-sample problems**: **Bias-Variance trade-off**.

- We make this trade-off **empirically**.

    \begin{align}
            min_{\beta} E(\beta) = \sum_{i=1}^n (y_i-\beta_0 - x_{i1}\beta_1 - \dots - x_{ip}\beta_p)^2 + \lambda \sum_{j=1}^p R(\beta_j)
    \end{align}

- **$\lambda$ is the price for this trade-off**.

How do we choose  $\lambda$?  $\rightarrow$  **Cross-validation**.


1. Initialization: Initially, we select a range of $\lambda$ values. These could be chosen based on prior knowledge, heuristics, or a predefined range.

2. Iteration Over Folds:

    - Fold 1 as Validation: We train the Ridge regression model on Folds 2-5 (combining them to form the training set) and validate the model on Fold 1. We record the MSE  for each $\lambda$  value.
    - Fold 2 as Validation: Next, we train on Folds 1, 3-5 and validate on Fold 2, recording the MSE  for each $\lambda$  value.
    - We repeat this process, training on four folds and validating on the remaining one, cycling through until each fold has served as the validation set.
  

<div>
<img src="../Week01/figs_notebook/fold.png" width="500"/>
</div>


- Error Aggregation: For each $\lambda$   value, we average the MSE across all five folds. 

- Selection of $\lambda$ : We select the $\lambda$  value that yields the lowest MSE. This is the value that we expect to generalize best to unseen data.

Let's do it on all the predictors:

In [ ]:
#Vector that needs predicting
y <- nlsy$lnw_2016

# Matrix of predictors (all but lnw_2016)
X <- as.matrix(nlsy  %>% select(-lnw_2016))


#### 1. Initialization: Initially, we select a range of $\lambda$ values. 

In [ ]:


# Define lambda sequence
lambda_seq <- 10^seq(4, -2, length = 100)
lambda_seq


#### 2. Iteration Over Folds:

##### Create the folds

In [ ]:
K <- 5  # 5-fold cross-validation

# Create folds
set.seed(123)  # Ensure reproducibility
folds <- sample(rep(1:K, length.out = length(y)))  # Assign randomly 1-5 to each observation.
folds 

# Prepare to loop

In [ ]:
# Store errors
cv_errors <- matrix(NA, nrow = length(lambda_seq), ncol = K)

# Loop over folds
for (k in 1:K) {

  test_idx <- which(folds == k) # Extract test indices for this fold
  
  # Training and testing folds
  X_train <- X[-test_idx, , drop = FALSE]  # Drop = FALSE prevents dimension reduction issues
  y_train <- y[-test_idx]
  X_test <- X[test_idx, , drop = FALSE]
  y_test <- y[test_idx]
  
  # Fit ridge regression for all lambdas (100 because that is the sequence we specified)
  ridge_model <- glmnet(X_train, y_train, alpha = 0, lambda = lambda_seq)
  
  # Predict on test set for each lambda
  y_pred <- predict(ridge_model, newx = X_test)
  
  # Compute MSE for each lambda
  cv_errors[, k] <- colMeans((y_test - y_pred)^2)
}


In [ ]:
cv_errors

##### Error Aggregation: For each $\lambda$   value, we average the MSE across all five folds. 

In [ ]:


# Average errors across folds
cv_mse <- rowMeans(cv_errors)

cv_mse

##### Selection of $\lambda$ : We select the $\lambda$  value that yields the lowest MSE. This is the value that we expect to generalize best to unseen data.

In [ ]:
# Optimal lambda (minimizing the cross-validation error)
lambda_opt <- lambda_seq[which.min(cv_mse)]
# Print optimal lambda
print(lambda_opt)


##### Train final model with optimal lambda

In [ ]:
final_model <- glmnet(X, y, alpha = 0, lambda = lambda_opt)

#### Easier way, use cv.glmnet 
(even easier, caret -> Next)

In [ ]:
# Cross-validation using cv.glmnet with the same folds
cv_ridge <- cv.glmnet(X, y, alpha = 0, lambda = lambda_seq, foldid = folds)
# foldid: an optional vector of values between 1 and nfolds identifying what fold each observation is in

In [ ]:
# Optimal lambda from cv.glmnet
lambda_opt_glmnet <- cv_ridge$lambda.min
print(lambda_opt_glmnet)

In [ ]:
cv_ridge

We can plot the MSE for each $\lambda$:

In [ ]:
plot(cv_ridge)

This plots the cross-validation curve (red dotted line) along with upper and lower standard deviation curves
along the $\lambda$ sequence (error bars). 

Two special values along the $\lambda$ sequence are indicated by the vertical dotted lines:
 - lambda.min is the value of $\lambda$ that gives minimum mean cross-validated error, while 
 - lambda.1se is the value of $\lambda$ that gives the most regularized model such that the cross-validated error is within one standard error of the minimum.

We can use the following code to get the value of `lambda.min` 

In [ ]:
cv_ridge$lambda.min

In [ ]:
log(cv_ridge$lambda.min)

and the model coefficients at that value of $\lambda$:

In [ ]:
head(coef(cv_ridge, s = "lambda.min"))

In practice, the 1SE rule is sometimes used because some studies found that models chosen by $\lambda_{1SE}$ **tend to perform better on test data** than those chosen by $\lambda_{\min}$ (Hastie et al., *The Elements of Statistical Learning*).

## Lasso

Now let's fit lasso, which can be fit using `glmnet()` with alpha = 1 and seeks to minimize:


$$
\frac{1}{2n}\sum_{i=1}^{n} \left( y_i - \beta_0 - \sum_{j=1}^{p} \beta_j x_{ij}    \right) ^ 2 + \lambda \sum_{j=1}^{p} |\beta_j|
$$

Notice that the intercept is not penalized. 


### No penalty = OLS

In [ ]:
lasso_no_pen <- glmnet(
  x = Xsmall,
  y = y,
  alpha = 1, #lasso
  lambda=0
)

In [ ]:
lasso_no_pen$beta

In [ ]:
summary(lm(y~Xsmall))

### With Penalty

In [ ]:
lasso_pen <- glmnet(
  x = Xsmall,
  y = y,
  alpha = 1, #lasso
  lambda=.02
)

In [ ]:
lasso_pen$beta

#### Larger Penalty

In [ ]:
lasso_pen_large <- glmnet(
  x = Xsmall,
  y = y,
  alpha = 1, #lasso
  lambda=1e70
)

In [ ]:
lasso_pen_large$beta

#### Various Penalties

In [ ]:
lasso01 <- glmnet(
  x = Xsmall,
  y = y,
  alpha = 1 #lasso
)

In [ ]:
plot(lasso01, xvar = "lambda")

### Penalty selection

In [ ]:
cv_lasso <- cv.glmnet(X, y, alpha = 1, lambda = lambda_seq, foldid = folds)

In [ ]:
plot(cv_lasso)

In [ ]:
cv_lasso$lambda.min

## Elastic Net 

Now let's fit EN, which can be fit using `glmnet()`  and seeks to minimize:

$$
\frac{1}{2n}\sum_{i=1}^{n} \left( y_i - \beta_0 - \sum_{j=1}^{p} \beta_j x_{ij}    \right) ^ 2 + \lambda\left[(1-\alpha)\frac{1}{2}\|\beta\|_2^2 + \alpha \|\beta\|_1\right],
$$

where 
- If $\alpha=1$ Lasso 
- If $\alpha=0$ Ridge 


- The elastic net simultaneously does automatic variable selection and continuous shrinkage, and it can select groups of correlated variables. 

- It is like a stretchable fishing net that retains ‘all the big fish’.

- Simulation studies and real data examples show that the elastic net often outperforms the lasso in terms of prediction accuracy

- The strict convexity part  of the penalty (ridge) solves the grouping instability problem 

- In **Elastic Net (EN)**, we have **two hyperparameters** to tune:
    1. **$\alpha$**: The mixing parameter between Lasso $(\alpha = 1)$ and Ridge $(\alpha = 0)$.
    2. **$\lambda$**: The penalty strength.
       
    - How to choose $(\lambda,\alpha)$? $\rightarrow$ Bidimensional Crossvalidation


To perform **bidimensional cross-validation** we can follow these steps:

1. **Choose a grid of $\alpha$ values**  from 0 to 1.
2. **For each $\alpha$, perform cross-validation over $\lambda$** using `cv.glmnet()`.
3. **Select the best $\alpha, \lambda$ pair** based on cross-validation error.



In [ ]:
# Define alpha grid (e.g., from 0 to 1 in steps of 0.2) #just to be fast, in practice, need a finer grid
alpha_grid <- seq(0, 1, by = 0.2)


# Initialize storage for results
cv_results <- data.frame(alpha = numeric(), lambda = numeric(), mse = numeric())

# Perform bidimensional cross-validation
for (alpha_value in alpha_grid) {
  # Perform cross-validation for each alpha
  cv_fit <- cv.glmnet(X, y, alpha = alpha_value, lambda = lambda_seq, foldid = folds) #note we are using the same folds and lambda sequences as before
  #how many specifications are we running?
    
  # Store best lambda and corresponding MSE
  best_lambda <- cv_fit$lambda.min
  best_mse <- min(cv_fit$cvm)  # Mean CV error
  
  # Append results
  cv_results <- rbind(cv_results, data.frame(alpha = alpha_value, lambda = best_lambda, mse = best_mse))
}


In [ ]:
cv_results

In [ ]:
# Find the best alpha-lambda combination
best_model <- cv_results[which.min(cv_results$mse), ]


# Print best alpha and lambda
print(paste("Best alpha:", best_model$alpha))
print(paste("Best lambda:", best_model$lambda))


In [ ]:
# Train the final model with the best alpha and lambda
final_model <- glmnet(X, y, alpha = best_model$alpha, lambda = best_model$lambda)

In [ ]:
# coef(final_model)

- **Elastic Net performs best when $\alpha$ is optimized** rather than arbitrarily chosen.
- **Allows flexibility between Lasso and Ridge**, automatically adjusting to the dataset.
- **Prevents overfitting or underfitting** by tuning both hyperparameters simultaneously.


# Caret

El paquete caret (abreviatura de **C**lassification **A**nd **RE**gression **T**raining) contiene funciones para agilizar el proceso de entrenamiento de modelos para problemas complejos de regresión y clasificación. Hay muchas funciones de modelado diferentes en `R`. Algunas tienen una sintaxis diferente para el entrenamiento y/o la predicción del modelo. El paquete comenzó como una forma de proporcionar una interfaz uniforme para las funciones en sí, así como una forma de estandarizar tareas comunes (como el ajuste de parámetros y la importancia de las variables).

El paquete contiene herramientas para:

  - división de datos.
  - preprocesamiento.
  - selección de características.
  - ajuste del modelo mediante remuestreo.
  - estimación de la importancia de cada variable.


In [ ]:
p_load("caret")

## Métodos de Remuestreo: Validación cruzada (Cross-validation)

Para modificar el método de remuestreo, se utiliza una función `trainControl.` La opción `method` controla el tipo de remuestreo y el valor predeterminado es `boot`. `cv`, se utiliza para especificar la validación cruzada en K-pates (K-fold-cross validation) y el argumento `repeats` controla el número de repeticiones. K está controlado por el argumento numérico y su valor predeterminado es 10. Por ejemplo, para hacer  una validación cruzada de 5 folds la síntaxis es:

In [ ]:
ctrl <- trainControl(
  method = "cv", 
  number = 5) # número de folds

Para incluir este nuevo método de remuestro se usa el argumento `trControl` en la función `train`como se muestra a continuación, en este caso usaremos todas las variables, salvo precio como variables independientes, para esto en vez del nombre de cada variable ponemos un punto después del tilde o virgulilla (`~`).


## Pre-procesando los datos

A menudo ciertos modelos requieren que los datos estén transformados o preprocesados antes de correr los modelos. Para especificar qué preprocesamiento debe ocurrir, la función de `train` tiene un argumento llamado `preProcess`. Este argumento toma distintas opciones:
  - `center` resta la media de los datos de cada variable predictiva a los valores de la variable,      
  - `scale` los divide por la desviación estándar. 
  

## Elastic Net

Con el algoritmo Elastic Net, debemos buscar dos hiperparámetros, $\lambda$ y $\alpha$, donde $\alpha$ define la combinación entre Lasso y Ridge. Podemos realizar esta búsqueda especificando el número de valores para la grilla mediante la función `tuneLength`.

In [ ]:
ENet <- train(lnw_2016~educ+mom_educ,
             data = nlsy,
             method = 'glmnet', 
             trControl = ctrl,
             tuneLength = 5) #specify the grid length

In [ ]:
ENet

In [ ]:
ggplot(ENet$results,  # Keep all results for plotting
       aes(x = lambda, y = RMSE, color = factor(alpha))) +
  geom_line() +
  geom_point(size = 2, alpha = 0.5) +
  scale_color_viridis_d(name = "Mixing Percentage (α)",
                        breaks = seq(0, 1, 0.1)) + # Only show multiples of 0.1 in legend
  labs(x = "Penalty (λ)", 
       y = "Root Mean Squared Error",
       title = "Elastic Net Performance Across Different Parameter Values") +
  theme_minimal() +
  theme(legend.position = "right")

In [ ]:
Enet_RMSE <- min(ENet$results$RMSE)
Enet_RMSE

In [ ]:
coef_Enet <- coef(ENet$finalModel, ENet$bestTune$lambda)
coef_Enet 